In [1]:
from flashbax.vault import Vault
vlt = Vault(
    vault_name="rware/tiny-4ag.vlt",
    vault_uid="Replay",
)

Loading vault found at /Users/callum/og-marl/vaults/rware/tiny-4ag.vlt/Replay


In [2]:
data = vlt.read()

In [4]:
data.experience.keys()

dict_keys(['action', 'done', 'legal_action_mask', 'observation', 'reward'])

In [5]:
import jax.numpy as jnp

def convert_timestep(input_dict):
    output_dict = {
        "observations": jnp.asarray(input_dict["observation"][..., 4:], dtype=jnp.float32),
        "rewards": input_dict["reward"],
        "actions": jnp.asarray(input_dict["action"], dtype=jnp.int32),
        "terminals": input_dict["done"],
        "truncations": jnp.zeros_like(input_dict["done"], dtype=jnp.bool_),
        "infos": {
            "legals": jnp.asarray(input_dict["legal_action_mask"], dtype=jnp.int32),
            "env_state": input_dict["observation"][..., 4:].reshape(*input_dict["observation"].shape[:2], -1)
        }
    }
    return output_dict

In [6]:
new_vault = Vault(
    vault_name="rware/tiny-4ag-converted.vlt",
    vault_uid="Replay",
    experience_structure=convert_timestep(data.experience),
)

New vault created at /Users/callum/og-marl/vaults/rware/tiny-4ag-converted.vlt/Replay
Since the provided buffer state has a temporal dimension of 6256, you must write to the vault at least every 6255 timesteps to avoid data loss.


In [7]:
from flashbax.buffers.trajectory_buffer import TrajectoryBufferState
import jax
new_vault.write(
    TrajectoryBufferState(
        experience=convert_timestep(data.experience),
        current_index=jax.tree_util.tree_flatten(data.experience)[0][0].shape[1],
        is_full=False,
    )
)

6256